In [ ]:
'''
Overview/ToDo:

Import Pandas, Numpy, and StringIO libraries
    Pandas      <-  data manipulation and analysis, provides data structures for efficiently storing and accessing data
    Numpy       <-  numerical computations, provides support for large multi-dimensional arrays and matrices
    StringIO    <-  read text data from memory

Reads 2 CSV files and 2 Excel files using the pd.read_csv() and pd.read_excel() functions

CSV files and Excel file are read into Pandas dataframes.

Use sheet_name parameter to specify the sheet name in the Excel file to be read

Renames the columns of the CSV dataframes using the rename() function
Map the negative and positive ion intensities by iterating over the rows of the dataframes
Check the conditions using the iterrows() function
Use the at() function to update the values of the df_pos dataframe

Removes undefined InChiKey values from the df_pos dataframe and reorder the rows
Use the astype() function and the < operator for re-ordering by InChiKey and MSI_level
Reorder the rows by InChiKey, MSI_level, and QC_RSD using the sort_values() function

Read in another Excel file into a Pandas dataframe and select a subset of columns using the [] operator

Reads another Excel file and select columns to keep from the reference library dataframe
Sort the reference library dataframe by InChiKey
Remove leading and trailing whitespaces from the InChiKey column of both dataframes
Remove duplicates from the reference library dataframe based on InChiKey using the drop_duplicates() function
Merges the reference library dataframe with the unknown dataframe based on InChiKey using the merge() function

Select columns to keep from the reference library dataframe and converts the Metabolite.name column to uppercase
Remove leading and trailing whitespaces
Remove duplicates from the reference library dataframe based on Metabolite.name using the drop_duplicates() function
Merge the reference library dataframe with the unknown dataframe based on Metabolite.name using the merge() function

Remove the QC_RSD column from the final dataframe
Concatenate the final dataframe with the unknown dataframe through the concat() function
Sort the final dataframe by MSI_level and InChiKey using the sort_values() function
Replace the MSI_level values of 2, 3, and 4 with 2A, 2B, and 2C respectively using the replace() function
Select the final columns to keep using the [] operator and write the final dataframe to an Excel file using the to_excel() function

Break down pipeline steps               ✓
Convert R code into Python Notebook     ✔
Successfully run locally                ✗
Run on VM                               ✘

'''

In [ ]:
'''
#   Attempt 1

import pandas as pd
import numpy as np
from io import StringIO

# Read CSV and Excel files
df_neg = pd.read_excel('C:/MetaboFilter-PythonFilter/Jon_Reverse_neg_052323-Example.xlsx', sheet_name='Know_compound_RT_match')
df_pos = pd.read_excel('C:/MetaboFilter-PythonFilter/Jon_Reverse_pos_052323-Example.xlsx', sheet_name='Know_compound_RT_match')
df1 = pd.read_csv('C:/MetaboFilter-PythonFilter/Hilic_IROA_IS_negative_lib.csv')
df2 = pd.read_csv('C:/MetaboFilter-PythonFilter/Reverse_IROA_IS_positive_lib.csv')

# Data preprocessing
df_neg = df_neg.rename(columns={'InChiKey': 'InChiKey', 'RT_inlib': 'RT_inlib', 'Average_Mz': 'Average_Mz', 'Average_Rt_min': 'Average_Rt_min', 'Metabolite.name': 'Metabolite_name'})
df1 = df1.rename(columns={'Metabolite_name': 'Metabolite_name', 'Average_Mz': 'Average_Mz', 'Average_Rt_min': 'Average_Rt_min', 'InChiKey': 'InChiKey'})
df_pos = df_pos.rename(columns={'InChiKey': 'InChiKey', 'RT_inlib': 'RT_inlib', 'Average_Mz': 'Average_Mz', 'Average_Rt_min': 'Average_Rt_min', 'Metabolite.name': 'Metabolite_name'})
df2 = df2.rename(columns={'Metabolite_name': 'Metabolite_name', 'Average_Mz': 'Average_Mz', 'Average_Rt_min': 'Average_Rt_min', 'InChiKey': 'InChiKey'})

# Mapping negative and positive ion intensities
for i, row in df_neg.iterrows():
    for j, row2 in df_pos.iterrows():
        if (row['InChiKey'] == row2['InChiKey']) and (np.abs(row['Average_Mz'] - row2['Average_Mz']) <= 0.005) and (np.abs(row['Average_Mz'] - row2['Average_Mz']) <= 0.005) and (np.abs(row['Average_Rt_min'] - row2['Average_Rt_min']) <= 0.3) and (np.abs(row['Average_Rt_min'] - row2['Average_Rt_min']) <= 0.3):
            df_pos.at[i, 'MSI_level'] = "1"
            df_pos.at[i, 'RT_inlib'] = df2['Average_Rt_min']

# Mapping MSI levels
for i in range(len(df_pos)):
    if df_pos.at[i, 'Total.score'] >= 95:
        df_pos.at[i, 'MSI_level'] = "2"

# Remove undefined InChiKey values
df_undefine = df_pos[~df_pos['InChiKey'].str.contains('undefined')]

# Reorder rows by InChiKey and MSI_level
df_undefine = df_undefine[df_undefine['InChiKey'].astype(str) < df_undefine['InChiKey'].astype(str)]

# Reorder rows by InChiKey, MSI_level, and QC_RSD
df_all = df_all[order(df_all['InChiKey'], df_all['MSI_level'], df_all['QC_RSD'])]

# read the dataset into a pandas dataframe
df_undefine = pd.read_excel('C:/MetaboFilter-PythonFilter/Tricore_Reaction_Reverse_neg_processed_R.xlsx', sheet_name='Unknown_compound')

# define the columns to keep
cols = ["Metabolite.name", "Average.Rt.min.", "Average.Mz", "Adduct.type", "MSI_level", "InChiKey", "QC_RSD", sample_col]

# subset the dataframe using only the selected columns
df_undefine = df_undefine[cols]

# read the reference library into a pandas dataframe
lb = pd.read_excel('C:/MetaboFilter-PythonFilter/Chemical Identification.xlsx', sheet_name='Sheet1')

# select the columns to keep from the reference library dataframe
lb_cols = ["InChiKey", "SMILES", "PubChem.CID", "KEGG.ID"]
lb_1 = lb[lb_cols]

# sort the reference library dataframe by InChiKey
lb_1 = lb_1.sort_values(by='InChiKey')

# remove leading and trailing whitespaces from the InChiKey column of both dataframes
lb_1['InChiKey'] = lb_1['InChiKey'].str.strip()
df_undefine['InChiKey'] = df_undefine['InChiKey'].str.strip()

# remove duplicates from the reference library dataframe based on InChiKey
lb_1 = lb_1.drop_duplicates(subset='InChiKey')

# merge the reference library dataframe with the unknown dataframe based on InChiKey
df_final = pd.merge(df_undefine, lb_1, on='InChiKey', how='left')

# select the columns to keep from the reference library dataframe
lb_cols = ["Metabolite.name", "SMILES", "PubChem.CID", "KEGG.ID"]
lb_2 = lb[lb_cols]

# convert the Metabolite.name column to uppercase and remove leading and trailing whitespaces
lb_2['Metabolite.name'] = lb_2['Metabolite.name'].str.upper()
lb_2['Metabolite.name'] = lb_2['Metabolite.name'].str.strip()
df_undefine['Metabolite.name'] = df_undefine['Metabolite.name'].str.strip()

# remove duplicates from the reference library dataframe based on Metabolite.name
lb_2 = lb_2.drop_duplicates(subset='Metabolite.name')

# merge the reference library dataframe with the unknown dataframe based on Metabolite.name
df_undefine = pd.merge(df_undefine, lb_2, on='Metabolite.name', how='left')

# remove the QC_RSD column from the final dataframe
df_final = df_final.drop(columns=['QC_RSD'])

# concatenate the final dataframe with the unknown dataframe
df_final = pd.concat([df_final, df_undefine], ignore_index=True)

# sort the final dataframe by MSI_level and InChiKey
df_final = df_final.sort_values(by=['MSI_level', 'InChiKey'])

# replace the MSI_level values of 2, 3, and 4 with 2A, 2B, and 2C respectively
df_final['MSI_level'].replace({'2': '2A', '3': '2B', '4': '2C'}, inplace=True)

# select the final columns to keep
col_format = ["Metabolite.name", "Average.Rt.min.", "Average.Mz", "Adduct.type", "MSI_level", "InChiKey", "SMILES", "PubChem.CID", "KEGG.ID", sample_col]
df_final = df_final[col_format]

# write the final dataframe to an excel file
writer = pd.ExcelWriter('C:/MetaboFilter-PythonFilter/Jon_Reverse_Reaction_combine_data_process.xlsx', engine='openpyxl')
df_final.to_excel(writer, sheet_name='final', index=False)
df_undefine.to_excel(writer, sheet_name='unknown', index=False)
writer.save()
'''

In [ ]:
'''#   Attempt 2

# Load required libraries
import pandas as pd
import re
#from openpyxl import load_workbook
#from openpyxl.utils.dataframe import dataframe_to_rows

# Read data from Excel files
df_neg = pd.read_excel('C:/Users/Metabolomics-Shortcut.lnk/MetaboFilter-PythonFilter/Jon_Reverse_neg_052323-Example.xlsx', sheet_name='Know_compound_RT_match')
df_pos = pd.read_excel('C:/Users/Metabolomics-Shortcut.lnk/MetaboFilter-PythonFilter/Jon_Reverse_pos_052323-Example.xlsx', sheet_name='Know_compound_RT_match')

# Read data from CSV file and assign column names
df1 = pd.read_csv('C:/Users/Metabolomics-Shortcut.lnk/MetaboFilter-PythonFilter/Hilic_IROA_IS_negative_lib.csv')
df1.columns = ['Metabolite_name', 'Average_Mz', 'Average_Rt_min', 'InChiKey']

# Update column names in df_neg
df_neg.rename(columns={'Average.Rt.min.': 'Average_Rt_min', 'Average.Mz': 'Average_Mz', 'Metabolite.name': 'Metabolite_name'}, inplace=True)

# Convert column values to appropriate data types
#df1['Average_Rt_min'] = pd.to_numeric(df1['Average_Rt_min'])
#df1.dtypes
df1['Average_Rt_min'] = pd.to_numeric(df1['Average_Rt_min'])

# Perform data transformations and matching
df_neg['Metabolite_name'] = df_neg['Metabolite_name'].str.replace(".", "")
df1['Metabolite_name'] = df1['Metabolite_name'].str.replace(".", "")
df_neg['Metabolite_name'] = df_neg['Metabolite_name'].str.upper()
df1['Metabolite_name'] = df1['Metabolite_name'].str.upper()
df_neg['Metabolite_name'] = df_neg['Metabolite_name'].str.strip()
df1['Metabolite_name'] = df1['Metabolite_name'].str.strip()

# Perform matching based on specific conditions
for i in range(len(df_neg)):
    for j in range(len(df1)):
        if (df_neg.loc[i, 'InChiKey'] == df1.loc[j, 'InChiKey'] and
                df_neg.loc[i, 'Average_Mz'] >= df1.loc[j, 'Average_Mz'] - 0.005 and
                df_neg.loc[i, 'Average_Mz'] <= df1.loc[j, 'Average_Mz'] + 0.005 and
                df_neg.loc[i, 'Average_Rt_min'] >= df1.loc[j, 'Average_Rt_min'] - 0.3 and
                df_neg.loc[i, 'Average_Rt_min'] <= df1.loc[j, 'Average_Rt_min'] + 0.3):
            
            df_neg.loc[i, 'MSI_level'] = "1"
            df_neg.loc[i, 'RT_inlib'] = df1.loc[j, 'Average_Rt_min']

# Update column names in df_neg
df_neg.rename(columns={'Average_Rt_min': 'Average.Rt.min.', 'Average_Mz': 'Average.Mz', 'Metabolite_name': 'Metabolite.name'}, inplace=True)

# Read data from CSV file into df2 and assign column names
df2 = pd.read_csv('C:/Users/Metabolomics-Shortcut.lnk/MetaboFilter-PythonFilter/Hilic_IROA_IS_positive_lib.csv')
df2.columns = ['Metabolite_name', 'Average_Mz', 'Average_Rt_min', 'InChiKey']
#df2.columns

# Set MSI_level and RT_inlib columns in df_pos
df_pos = df_pos.assign(MSI_level="", RT_inlib="")
df_pos = df_pos.reindex(columns=['MSI_level', 'RT_inlib'] + df_pos.columns[:-2].tolist())

# Update column names in df_pos
df_pos.rename(columns={"Average.Rt.min.": "Average_Rt_min", "Average_Mz": "Average_Mz", "Metabolite.name": "Metabolite_name"}, inplace=True)
print(df_pos.columns)

# Convert column values in df2 to appropriate data types
df2['Average_Rt_min'] = pd.to_numeric(df2['Average_Rt_min'])
print(df2.dtypes)

# Perform string transformations on Metabolite_name columns in df_pos and df2
df_pos['Metabolite_name'] = df_pos['Metabolite_name'].str.replace("[.]", "")
df2['Metabolite_name'] = df2['Metabolite_name'].str.replace("[.]", "")
df_pos['Metabolite_name'] = df_pos['Metabolite_name'].str.upper()
df2['Metabolite_name'] = df2['Metabolite_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df2['Metabolite_name'] = df2['Metabolite_name'].str.upper()
df_pos['Metabolite_name'] = df_pos['Metabolite_name'].str.strip()
df2['Metabolite_name'] = df2['Metabolite_name'].str.strip()

# Perform matching between df_pos and df2 based on specific conditions
for i in range(len(df_pos)):
    for j in range(len(df2)):
        if (df_pos.loc[i, 'InChiKey'] == df2.loc[j, 'InChiKey']) and /
                (df_pos.loc[i, 'Average_Mz'] >= df2.loc[j, 'Average_Mz'] - 0.005) and /
                (df_pos.loc[i, 'Average_Mz'] <= df2.loc[j, 'Average_Mz'] + 0.005) and /
                (df_pos.loc[i, 'Average_Rt_min'] >= df2.loc[j, 'Average_Rt_min'] - 0.3) and /
                (df_pos.loc[i, 'Average_Rt_min'] <= df2.loc[j, 'Average_Rt_min'] + 0.3):
            df_pos.loc[i, 'MSI_level'] = "1"
            df_pos.loc[i, 'RT_inlib'] = df2.loc[j, 'Average_Rt_min']

# Update column names in df_pos
df_pos.rename(columns={"Average_Rt_min": "Average.Rt.min.", "Average_Mz": "Average_Mz", "Metabolite_name": "Metabolite.name"}, inplace=True)
df_pos.columns = df_neg.columns

# Combine df_neg and df_pos into a single dataframe, df
df = pd.concat([df_neg, df_pos])

# Sort df by MSI_level in descending order
df = df.sort_values(by='MSI_level', ascending=False)

# Create MSI_1 dataframe containing rows with MSI_level = '1'
MSI_1 = df[df['MSI_level'] == '1']

# Create df_1 dataframe containing rows with MSI_level not equal to '1'
df_1 = df[df['MSI_level'] != '1']

# Convert Total.score column in df_1 to numeric
df_1['Total.score'] = pd.to_numeric(df_1['Total.score'])

# Update MSI_level values in df_1 based on Total.score condition
df_1.loc[df_1['Total.score'] >= 95, 'MSI_level'] = '2'

# Sort df_1 by MSI_level in descending order
df_1 = df_1.sort_values(by='MSI_level', ascending=False)

# Create MSI_2 dataframe containing rows with MSI_level = '2'
MSI_2 = df_1[df_1['MSI_level'] == '2']

# Create df_2 dataframe containing rows with MSI_level not equal to '2'
df_2 = df_1[df_1['MSI_level'] != '2']

# Update MSI_level values in df_2 based on Total.score condition
df_2.loc[df_2['Total.score'] >= 90, 'MSI_level'] = '3'

# Update MSI_level values in df_2 not equal to '3' to '4'
df_2.loc[df_2['MSI_level'] != '3', 'MSI_level'] = '4'

# Combine MSI_1, MSI_2, and df_2 into dt_all dataframe
dt_all = pd.concat([MSI_1, MSI_2, df_2])

# Remove trailing spaces in InChiKey column
dt_all['InChiKey'] = dt_all['InChiKey'].str.strip()

# Separate dt_all into dt_undefine and dt_all based on InChiKey containing "undefined"
dt_undefine = dt_all[~dt_all['InChiKey'].str.contains('undefined')]
dt_all = dt_all[dt_all['InChiKey'].str.contains('undefined')]

# Sort dt_all by InChiKey, MSI_level, and QC_RSD
dt_all = dt_all.sort_values(by=['InChiKey', 'MSI_level', 'QC_RSD'])

# Write dt_all to a CSV file named "remove_rep.csv"
dt_all.to_csv('remove_rep.csv', index=False)

# Remove duplicate rows based on InChiKey in dt_all and create final dataframe
final = dt_all.drop_duplicates(subset='InChiKey')

# Perform string transformations and sorting on dt_undefine
dt_undefine['Metabolite.name'] = dt_undefine['Metabolite.name'].str.upper()
dt_undefine['Metabolite.name'] = dt_undefine['Metabolite.name'].str.replace("[.]", "")
dt_undefine['Metabolite.name'] = dt_undefine['Metabolite.name'].str.strip()
dt_undefine = dt_undefine.sort_values(by=['Metabolite.name', 'MSI_level', 'QC_RSD'])
dt_undefine = dt_undefine.drop_duplicates(subset='Metabolite.name')

# Define the column range for samples in dt_all
start = dt_final.columns.get_loc("MS.MS.spectrum") + 1
end = dt_final.columns.get_loc('Avg') - 1
sample_col = dt_final.columns[start:end]

# Define the columns to select for dt_undefine and data
cols = ['Metabolite.name', 'Average.Rt.min.', 'Average.Mz', 'Adduct.type', 'MSI_level', 'InChiKey', 'QC_RSD'] + list(sample_col)
dt_undefine = dt_undefine[cols]
data = final[cols]

# Read "Chemical Identification.xlsx" file into lb dataframe
lb = pd.read_excel('C:/Users/Metabolomics-Shortcut.lnk/MetaboFilter-PythonFilter/Chemical Identification.xlsx', sheet_name='Sheet1')

# Select specific columns from lb dataframe
lb_1 = lb[['InChiKey', 'SMILES', 'PubChem.CID', 'KEGG.ID']]

# Remove duplicate rows based on InChiKey in lb_1
lb_1 = lb_1.sort_values(by='InChiKey')
lb_1['InChiKey'] = lb_1['InChiKey'].str.strip()
lb_1 = lb_1.drop_duplicates(subset='InChiKey')

# Merge data and lb_1 dataframes based on InChiKey, keeping all rows from data
dt_final = pd.merge(data, lb_1, on='InChiKey', how='left')

# Remove QC_RSD column from dt_final
dt_final = dt_final.drop(columns='QC_RSD')

# Select specific columns from lb dataframe
lb_2 = lb[['Metabolite.name', 'SMILES', 'PubChem.CID', 'KEGG.ID']]

# Perform string transformations and sorting on lb_2 and dt_undefine
lb_2['Metabolite.name'] = lb_2['Metabolite.name'].str.upper()
lb_2['Metabolite.name'] = lb_2['Metabolite.name'].str.strip()
dt_undefine['Metabolite.name'] = dt_undefine['Metabolite.name'].str.strip()
lb_2 = lb_2.drop_duplicates(subset='Metabolite.name')

# Merge dt_undefine and lb_2 dataframes based on Metabolite.name, keeping all rows from dt_undefine
dt_undefine = pd.merge(dt_undefine, lb_2, on='Metabolite.name', how='left')

# Remove QC_RSD column from dt_undefine
dt_undefine = dt_undefine.drop(columns='QC_RSD')

# Combine dt_final and dt_undefine into dt_final
dt_final = pd.concat([dt_final, dt_undefine])

# Display column names of dt_final
print(dt_final.columns)

# Sort dt_final by MSI_level
dt_final = dt_final.sort_values(by='MSI_level')

# Update MSI_level values in dt_final
dt_final.loc[dt_final['MSI_level'] == '2', 'MSI_level'] = '2A'
dt_final.loc[dt_final['MSI_level'] == '3', 'MSI_level'] = '2B'
dt_final.loc[dt_final['MSI_level'] == '4', 'MSI_level'] = '2C'

# Define the column format for dt_final
col_format = ['Metabolite.name', 'Average.Rt.min.', 'Average.Mz', 'Adduct.type', 'MSI_level', 'InChiKey', 'SMILES', 'PubChem.CID', 'KEGG.ID'] + list(sample_col)
dt_final = dt_final[col_format]

# Sort dt_final by MSI_level and InChiKey
dt_final = dt_final.sort_values(by=['MSI_level', 'InChiKey'])

# Read "Tricore_Reaction_Reverse_neg_processed_R.xlsx" or similar file into unknown1 dataframe
unknown1 = pd.read_excel('C:/Users/Metabolomics-Shortcut.lnk/MetaboFilter-PythonFilter/Jon_Reverse_neg_052323-Example.xlsx', sheet_name='Unknown_compound')

# Read "Tricore_Reaction_Reverse_pos_processed_R.xlsx" or similar file into unknown2 dataframe
unknown2 = pd.read_excel('C:/Users/Metabolomics-Shortcut.lnk/MetaboFilter-PythonFilter/Jon_Reverse_pos_052323-Example.xlsx', sheet_name='Unknown_compound')

# Set column names of unknown2 to match unknown1
unknown2.columns = unknown1.columns

# Combine unknown1 and unknown2 into unknown dataframe
unknown = pd.concat([unknown1, unknown2])

# Select specific columns from unknown dataframe
unknown = unknown[['Metabolite.name', 'Average.Rt.min.', 'Average.Mz', 'Adduct.type'] + list(sample_col)]

# Set the value of "Metabolite.name" column in unknown dataframe to 'Unknown'
unknown['Metabolite.name'] = 'Unknown'

# Create a dictionary of datasets with names
dataset_names = {'Known_compound': dt_final, 'Unknown_compound': unknown}

# Write dataset_names to "Ω_combine_data_process.xlsx" file using openpyxl package
'''
#with pd.ExcelWriter('C:\MetaboFilter-PythonFilter\Jon_Reverse_Reaction_combine_data_process.xlsx', engine='openpyxl') as writer:
#    for name, dataset in dataset_names.items():
#        dataset.to_excel(writer, sheet_name=name, index=False)

#    writer.save()
'''
# Write each dataset in dataset_names to a new sheet in the Excel file specified by filename.
# Define the filename of the Excel file
filename = 'C:/Users/Metabolomics-Shortcut.lnk/MetaboFilter-PythonFilter/Jon_Reverse_Reaction_combine_data_process.xlsx'

# Create a Pandas Excel writer
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Iterate over each dataset in dataset_names
for name, dataset in dataset_names.items():
    # Write the dataset to a new sheet in the Excel file
    dataset.to_excel(writer, sheet_name=name, index=False)

# Save the Excel file
writer.save()

'''

In [ ]:
#   Attempt 3
#   Python 3.9.1
import sys
import os
sys.path
print(1)
# Load required libraries
#pip install wheel
#pip install pandas
#py -m pip install pandas
import pandas as pd
import re
print(2)
#pip install selenium
#pip3 uninstall selenium
#pip3 install selenium
import selenium
#!{sys.executable} -m pip install openpyxl
#!pip install openpyxl
#!pip3 install openpyxl
#python -m pip3 install openpyxl
import openpyxl
print(3)
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
print(4)

In [3]:
# Read CSV file
data = pd.read_csv('Reverse_IROA_IS_positive_lib.csv')

# Remove rows with missing values in the MS.MS.spectrum column
data = data[~data['MS.MS.spectrum'].isna()]

# Extract columns containing Avg values
cols = [col for col in data.columns if 'Avg' in col]

# Remove rows containing "w/o MS2" or "w/o MS2:" in the Metabolite.name column
data = data[~((data['Metabolite.name'].str.contains('w/o MS2')) | (data['Metabolite.name'].str.contains('w/o MS2:')))]

# Separate ISTD, CUDA, and other metabolites
ISTD = data[data['Metabolite.name'].str.contains('iSTD')]
CUDA = data[data['Metabolite.name'].str.contains('CUDA')]
ISTD = pd.concat([CUDA, ISTD])

# Remove rows containing "w/o MS2" or "w/o MS2:" in the Metabolite.name column
ISTD = ISTD[~((ISTD['Metabolite.name'].str.contains('w/o MS2')) | (ISTD['Metabolite.name'].str.contains('w/o MS2:')))]

# Process data to calculate fold change and RSD
data['RSD'] = data[cols].std(axis=1) / data[cols].mean(axis=1) * 100
data['fold'] = data[cols].max(axis=1) / data[cols].mean(axis=1) * 100

# Filter data based on fold change and QC_RSD
sort_data = data[(data['fold'] >= 10) & (data['QC_RSD'] <= 30)].sort_values(by='QC_RSD', ascending=False)

# Separate data into known and unknown compounds
data_unknown = sort_data[sort_data['Metabolite.name'].str.contains('Unknown|w/o MS2', na=False)]
data_known = sort_data[~sort_data['Metabolite.name'].str.contains('Unknown|w/o MS2', na=False)]
data_known['MS.MS.spectrum'] = data_known['MS.MS.spectrum'].replace('None', '')

# Merge with internal standard data
ISTD = pd.concat([data_unknown, data_known])

# Write processed data to an Excel file
wb = load_workbook('Tricore_Reaction_Reverse_pos_processed_R.xlsx')
ws = wb.create_sheet('Processed_data')
with pd.ExcelWriter('Tricore_Reaction_Reverse_pos_processed_R.xlsx', engine='openpyxl') as writer:
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)
    ISTD.to_excel(writer, sheet_name='Processed_data', index=False)
    writer.save()


FileNotFoundError: [Errno 2] No such file or directory: 'Tricore_Reaction_reverse_pos.csv'